# 导入模块

In [ ]:
from selenium import webdriver
import time
import os
import re
import requests

# 创建数据库设置身高星座年月日等数据列表

In [1]:
from pony.orm import *


db = Database()


class Star(db.Entity):
    id = PrimaryKey(int, auto=True)
    name = Optional(str, column='name')
    picture=Optional(str, column='picture')
    href1=Optional(str, column='href1')
    gender = Optional(str, column='gender')
    href = Optional(str, column='href')
    nian = Optional(int, column='nian')
    yue = Optional(int, column='yue')
    ri= Optional(int, column='ri')
    xz = Optional(str, column='xz')
    html = Optional(str, column='html')
    address = Optional(str, column='address')
    sg = Optional(float, column='sg')



# db.generate_mapping()

ModuleNotFoundError: No module named 'pony'

In [ ]:
dbpath = 'd:/mingxing/star.sqlite'
if os.path.exists(dbpath):
    os.remove(dbpath)
f = open(dbpath,"w")
f.close()

In [ ]:
db.bind(provider='sqlite',filename='d:/mingxing/star.sqlite')

db.generate_mapping(create_tables=True)

set_sql_debug(True)

# 定义把名字、链接、图片，入库函数

In [ ]:
def get_data(h):
    reobj = re.compile(r'<div class="op_exactqa_item c-gap-bottom c-span4 [\d\D]*?"><div class="op_exactqa_feedback OP_LOG_BTN">[\d\D]*?<a href="([\d\D]*?)" title="([\d\D]*?)" target="_blank"><img class="[\d\D]*?" src="([\d\D]*?)">')
    for match in reobj.finditer(h):
        href2=match.group(1)
        name1=match.group(2)
        picture1=match.group(3)
        s=Star(name=name1,href1=href2,picture=picture1)
        db.commit()

# 定义获取明星百科链接并入库函数

In [ ]:
def get_href(n):
    star.href="https://baike.baidu.com/item/"+n
    time.sleep(2)
    db.commit()

#  定义获取明星详情页链接的源代码入库函数

In [ ]:
def get_html(h):
    head = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
    r = requests.get(h,headers = head)
    r.encoding = 'utf-8'
    star.html = r.text
    db.commit()

# 定义获取明星的星座并入库函数

In [ ]:
def get_xz(h):
    if re.search(r'座</dt>\n<dd class="basicInfo-item value">\n<a[\d\D]*?">([\d\D]*?)</a',h):
        match = re.search(r'座</dt>\n<dd class="basicInfo-item value">\n<a[\d\D]*?">([\d\D]*?)</a',h)
        xz1 = match.group(1)
        star.xz = xz1
        time.sleep(2)
        db.commit()
    elif re.search(r'座</dt>\n<dd class="basicInfo-item value">\n([\d\D]*?)\n<',h):
        match = re.search(r'座</dt>\n<dd class="basicInfo-item value">\n([\d\D]*?)\n<',h)
        xz1 = match.group(1)
        star.xz = xz1
        time.sleep(2)
        db.commit()

# 定义获取明星的身高并入库的函数

In [ ]:
def get_sg(h):
    if re.search(r'高</dt>\n<dd class="basicInfo-item value">\n([\d\D]*?)\n<',h):
        height = re.search(r'高</dt>\n<dd class="basicInfo-item value">\n([\d\D]*?)\n<',h).group(1).strip()
        try:
            if ("cm" in height) or ("CM" in height) or ("Cm" in height) or ("厘米" in height):
                height = float(height[:-2])
            elif  ("m" in height) or ("M" in height) or ("米" in height):
            height = float(height[:-1])*100
            star.height=height
            time.sleep(2)
            db.commit()

# 定义获取明星的性别、出生地并入库的函数

In [ ]:
def get_gender():
    star.gender="女"
    time.sleep(2)
    db.commit()
    
def get_address(h):
    if re.search(r'<dt class="basicInfo-item name">出生地[\d\D]*?">([\d\D]*?)<',h):
        match = re.search(r'<dt class="basicInfo-item name">出生地[\d\D]*?">([\d\D]*?)<',h)
        jg1=match.group(1)
        star.address=jg1
        time.sleep(2)
        db.commit()

# 定义获取明星的出生年，月，日并入库的函数

In [ ]:
def get_birth(h):
    if  re.search(r'<dt class="basicInfo-item name">出生日期[\d\D]*?">([\d\D]*?)</dd>',h):
        birth = re.search(r'<dt class="basicInfo-item name">出生日期[\d\D]*?">([\d\D]*?)</dd>',h).group(1).strip()
        if "年" in birth and "月" in birth and "日" in birth: 
            year,month,day = re.split("[年月日]",birth)[:3]
            star.year = int(year)
            star.month = int(month)
            star.day = int(day)
            db.commit

# 整体步骤

In [ ]:
driver=webdriver.Chrome()
url = 'https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&rsv_idx=2&tn=baiduhome_pg&wd=%E6%98%8E%E6%98%9F&rsv_spt=1&oq=%25E6%2598%258E%25E6%2598%259F&rsv_pq=a1a2cf770002ec2f&rsv_t=0e3baM8Xu8qPUVhj0snvRVkEdX1Le%2Bm%2FZvXgmFXh1EnnOrdvEFvLRImf6oviVMQdrqmk&rqlang=cn&rsv_enter=0&rsv_dl=tb'
driver.get(url)
time.sleep(2)
driver.find_element_by_xpath("//div[@id='1']/div/div/div/div[2]/p/span[4]").click()
time.sleep(2)
driver.find_element_by_xpath("//div[@id='1']/div/div/div/div[2]/p[2]/span[3]").click()

In [ ]:
for i in range(330):
    html = driver.page_source
    get_data(html)
    with db_session:
        for star in Star.select():
            get_href(star.name)
            get_html(star.href)
            get_xz(star.html)
            get_sg(star.html)
            get_gender()
            get_address(star.html)
            get_birth(star.html)